In [ ]:
import asyncio
from polymarket_dashboard.src.polymarket_dashboard.src.db.ingest_stream import parse_message

conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()


async def main(conf, queue, symbols, table_name):
    executor = ProcessPoolExecutor(max_workers=10)
    db_task = asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor))
    await binance_price_ws(queue=queue, symbols=symbols)
    await queue.put(None)
    await db_task

await main(conf, queue, symbols, table_name)

In [5]:
from polymarket_dashboard.src.polymarket_dashboard.src.config.logger import logger
import asyncio
from polymarket_dashboard.src.polymarket_dashboard.src.stream import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket_dashboard.src.polymarket_dashboard.src.db.ingest_stream import insert_into_db
from polymarket_dashboard.src.polymarket_dashboard.src.transform import parse_binance_message
conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()

async def main():

    executor = ThreadPoolExecutor(max_workers=10)
    producers = [asyncio.create_task(binance_price_ws(queue=queue, symbols=symbols)),]
    workers = [asyncio.create_task(insert_into_db(conf, table_name, queue, parse_binance_message, executor, logger))]

    await asyncio.gather(*producers, *workers)

await main()

2026-01-18 14:58:24,518 | INFO | Starting DB insert task...


Connected to Binance Futures Ticker WebSocket!


2026-01-18 14:58:26,580 | INFO | Received message: ('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768744706504, 's': 'BTCUSDT', 'p': '-266.40', 'P': '-0.279', 'w': '95193.25', 'c': '95103.40', 'Q': '1.386', 'o': '95369.80', 'h': '95621.90', 'l': '94819.50', 'v': '44953.644', 'q': '4279283387.37', 'O': 1768658280000, 'C': 1768744706501, 'F': 7111221161, 'L': 7112138976, 'n': 913107})


('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768744706504, 's': 'BTCUSDT', 'p': '-266.40', 'P': '-0.279', 'w': '95193.25', 'c': '95103.40', 'Q': '1.386', 'o': '95369.80', 'h': '95621.90', 'l': '94819.50', 'v': '44953.644', 'q': '4279283387.37', 'O': 1768658280000, 'C': 1768744706501, 'F': 7111221161, 'L': 7112138976, 'n': 913107})


TypeError: tuple indices must be integers or slices, not str

{'C': 1768744706501,
 'E': 1768744706504,
 'F': 7111221161,
 'L': 7112138976,
 'O': 1768658280000,
 'P': '-0.279',
 'Q': '1.386',
 'c': '95103.40',
 'e': '24hrTicker',
 'h': '95621.90',
 'l': '94819.50',
 'n': 913107,
 'o': '95369.80',
 'p': '-266.40',
 'q': '4279283387.37',
 's': 'BTCUSDT',
 'v': '44953.644',
 'w': '95193.25'}
